In [1]:
#Importing necessary libraries
from sagemaker.huggingface import HuggingFace
import sagemaker
import os
from dotenv import load_dotenv
load_dotenv()

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/abhijitdeshpande/Library/Application Support/sagemaker/config.yaml


True

In [3]:
#settinng up AWS profile
os.environ['AWS_PROFILE'] = 'dev-user'

session = sagemaker.Session()

In [4]:
#Defining hyper parameters
hyper_parameters = {
    'model_id': 'google/flan-t5-base',
    'rank': 64,
    'alpha': 128,
    'dropout': 0.05,
    'bias': 'none',
    'lr': 1e-5,
    'epochs': 10,
    'wd': 0.01,
    'logging_steps': 50,
    'batch_size': 32,
    'save_steps': 200,
    'eval_steps': 100,
    'target_module': 'q,k,v,o',
    'early_stopping': 3
}

In [5]:
huggingface_estimator = HuggingFace(
            entry_point='train.py',
            source_dir=os.getenv("SOURCE_DIR"),
            role=os.getenv("ROLE"),
            instance_type='ml.g4dn.xlarge',
            instance_count=1,
            transformers_version='4.49.0',
            py_version='py311',
            pytorch_version='2.5.1',
            hyperparameters = hyper_parameters
)

In [16]:
# # upload data to S3
# input_data = session.upload_data(path='data/data_full.json',bucket='gen-ai-repository', key_prefix='finetuning/flan-t5/data')

In [6]:
huggingface_estimator.fit({'training': 's3://gen-ai-repository/finetuning/flan-t5/data/'})

INFO:sagemaker.image_uris:image_uri is not presented, retrieving image_uri based on instance_type, framework etc.
INFO:sagemaker:Creating training-job with name: huggingface-pytorch-training-2025-06-28-15-47-21-142


2025-06-28 15:47:23 Starting - Starting the training job...
2025-06-28 15:47:50 Starting - Preparing the instances for training...
2025-06-28 15:48:22 Downloading - Downloading input data...
2025-06-28 15:48:47 Downloading - Downloading the training image...........................
2025-06-28 15:53:25 Training - Training image download completed. Training in progress..bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
CUDA compat package should be installed for NVIDIA driver smaller than 550.144.03
Current installed NVIDIA driver version is 550.163.01
Skipping CUDA compat setup as newer NVIDIA driver is installed
2025-06-28 15:53:38,458 sagemaker-training-toolkit INFO     Imported framework sagemaker_pytorch_container.training
2025-06-28 15:53:38,476 sagemaker-training-toolkit INFO     No Neurons detected (normal if no neurons installed)
2025-06-28 15:53:38,486 sagemaker_pytorch_container.training INFO     Block until all hos

In [7]:
model_artifact_s3_uri = huggingface_estimator.model_data
print("Model Artifact S3 URI:")
print(model_artifact_s3_uri)

Model Artifact S3 URI:
s3://sagemaker-us-east-1-720332985926/huggingface-pytorch-training-2025-06-28-15-47-21-142/output/model.tar.gz
